# Agents and Tools

In this section of the lab, we will review some agent concepts and see how agents can interact the basic tools

- Define a basic agent, using the `builtin::websearch` tool from Llama Stack
- Get familiar with concepts like "Turns" and "Steps" and "Tool_Calling" that the agent takes
- Run the agent to return web results
- Optional: experience a prompt chaining example

> **Note:**
>This section is intended only as an introduction to Agent concepts and language, and we will get into more exciting work in the next modules. If you are familiar with these concepts, feel free to skip this section.
> 


> **Note:**
> For the sake of simplicity and time, we will skip through some of the python basics we went through in previous modules and treat this more like a nomral python application 
> 

### Install Python Prerequisist

As always, let's start by installing the Python Libraries we neeed, we will add some additional libraries to support some of our tools. 

In [1]:
!pip install -U llama-stack-client==0.2.5 dotenv geocoder


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


### Define the LLamastack server and Model

Let's define the libraries we need and our basic Llama Stack configurations:

In [2]:
import os

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient

# These libraries are just here to print the results from the agent in a more human-readable way 
from src.utils import step_printer
from termcolor import cprint
import uuid
from llama_stack_client.lib.agents.event_logger import EventLogger
stream=False ## Defaulting to False, you can change this throughout the section to "True" if you wanted to see the output in another format (Using EventLogger)

# for our lab, we will just define our variables manualy here, in a regular application, this would be ready directly from the local .env file and we would comment these lines out
os.environ['LLAMA_STACK_SERVER'] = 'http://localhost:8321'

# We will be using the Tavily web search service (docs.tavily.com/)
tavily_search_api_key='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'
provider_data = {"tavily_search_api_key": tavily_search_api_key}

> **Note:**
>When running this code in a regular Python application, we would usually like to read environment variables from an `.env` file, for our needs in this lab, we will hard code these in this cell, to make things more clear
>

### Initialize the *Client* 
We will not define our client, and introduce a new pattern/trick that will help us later when we try to move between models and other providers within Llama Stack. 

In [3]:
LLAMA_STACK_SERVER=os.getenv("LLAMA_STACK_SERVER")

client = LlamaStackClient(
    base_url=LLAMA_STACK_SERVER,
    provider_data=provider_data
)
# List available models
models = client.models.list()
print("--- Available models: ---")
for m in models:
    print(f"{m.identifier} - {m.provider_id} - {m.provider_resource_id}")

# For our development purposes, we might want to change different models and test with them, these lines select the first available model in the Llama Stack server. 
SELECTED_MODEL=models[0].identifier
print("--- Selected model: ---")
print(SELECTED_MODEL)

--- Available models: ---
meta-llama/Llama-3.2-3B-Instruct - ollama - llama3.2:3b-instruct-fp16
all-MiniLM-L6-v2 - ollama - all-minilm:latest
--- Selected model: ---
meta-llama/Llama-3.2-3B-Instruct


### Define the a single tool agent
Here we define our new agent, we will pass along the `client` defining our connection to the Llama Stack server, the `model` we selected in the previous step, the `tools` array with all available tools for the agent, and crucually, the prompt, named `instructions` for the agent.

> **Note:**
> You might notice that we are being very specific in our prompt, for the purpose of lab we are using rather small models and are providing a narrow path to follow, in a real world scenario, prompts and LLMs can be experimented with to produce the best results.
> 

In [4]:
from llama_stack_client.lib.agents.agent import Agent

agent = Agent(
    client, 
    model=SELECTED_MODEL,
    instructions="""You are a helpful websearch assistant. When you are asked to search the latest you must use a tool. 
            Whenever a tool is called, be sure return the response in a friendly and helpful tone.
            """ ,
    tools=["builtin::websearch"],
)

### inspecting the Agent's response structure

Let's give our agent a couple of easy questions to see how it calls our `builtin::websearch` tool.
Notice that we are generating a new session ID for each question as convesation memory is shared throughtout the session. (More on this later!)


In [5]:
user_prompts = [
       "search the web for the latest in OpenShift?",
       "search for the current weather in boston",
]


for prompt in user_prompts:
    # Generate a new Unique Identifier for each session 
    new_uuid = uuid.uuid4()
    session_id = agent.create_session(f"web-session-{new_uuid}")

    cprint(f"\n{'='*100}\nProcessing user query: {prompt}\n{'='*100}", "blue")

    
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: search the web for the latest in OpenShift?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'latest news OpenShift'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'latest news OpenShift',
│   'top_k': [
│   │   {
│   │   │   'title': 'Red Hat unveils OpenShift 4.18: Enhanced security and virtualization ...',
│   │   │   'url': 'https://www.redhat.com/en/blog/what-you-need-to-know-red-hat-openshift-418',
│   │   │   'content': 'The latest release introduces cert-manager Operator for Red Hat OpenShift v1.15 with technology preview of the powerful istio-csr agent, enabling seamless integration to secure Red Hat OpenShift Service Mesh workload and control plane components. With the istio-csr agent, the istio agent running on OpenShift Service Mesh retrieves certificates',
│   │   │   'score': 0.7746691,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Azure Red Hat OpenShift: April 2025 Update | Microsoft Community Hub',
│   │   │   'url': 'https://techcommunity.microsoft.com/blog/appsonazureblog/azure-red-hat-openshift-april-2025-update/4400331',
│   │   │   'content': "OpenShift 4.16 on Day 0 → Now available for new ARO cluster installs with the latest features and improvements. See what's new. Note that a cluster version upgrade support (Day 2 operation) kicks in for existing clusters as soon as an upgrade path is available. This happens shortly after OCP GA availability for a version.",
│   │   │   'score': 0.769098,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Experience the many enhancements in OpenShift 4.18',
│   │   │   'url': 'https://developers.redhat.com/blog/2025/02/25/whats-new-developers-red-hat-openshift-418',
│   │   │   'content': 'Red Hat Enhances Security and Virtualization Experience with Latest Version of Red Hat OpenShift . Red Hat OpenShift Container Platform 4.18 release notes. Get started: Start your OpenShift journey with your Developer Sandbox. Find resources for Getting Started with OpenShift. Discover more ways to get started by downloading Red Hat OpenShift.',
│   │   │   'score': 0.7251239,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Red Hat OpenShift 4.17: What you need to know',
│   │   │   'url': 'https://www.redhat.com/en/blog/what-you-need-to-know-red-hat-openshift-417',
│   │   │   'content': 'Red Hat OpenShift 4.17 is now generally available. Based on Kubernetes 1.30 and CRI-O 1.30, OpenShift 4.17 features expanded control plane options, increased flexibility for virtualization and networking, new capabilities to leverage generative AI, and continued investment in Red Hat OpenShift Platform Plus.These additions further accelerate innovation with OpenShift without compromising on',
│   │   │   'score': 0.6954834,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': "What's new in Red Hat OpenShift",
│   │   │   'url': 'https://www.redhat.com/en/whats-new-red-hat-openshift',
│   │   │   'content': 'Red Hat AI A portfolio for developing and deploying artificial intelligence solutions across the hybrid cloud.; Red Hat Enterprise Linux A flexible, stable operating system to support hybrid cloud innovation.; Red Hat OpenShift A container platform to build, modernize, and deploy applications at scale.; Red Hat Ansible Automation Platform A foundation for implementing enterprise-wide automation.',
│   │   │   'score': 0.42935818,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Here's the final answer to the user's question:

The latest news on OpenShift includes:

* Red Hat unveils OpenShift 4.18 with enhanced security and virtualization features
* Azure Red Hat OpenShift: April 2025 Update is now available for new ARO cluster installs with the latest features and improvements
* Experience the many enhancements in OpenShift 4.18, which includes expanded control plane options, increased flexibility for virtualization and networking, and new capabilities to leverage generative AI
* Red Hat OpenShift 4.17 is now generally available, featuring expanded control plane options, increased flexibility for virtualization and networking, and new capabilities to leverage generative AI

These updates aim to accelerate innovation with OpenShift without compromising on security and performance.

========== Query processing completed ========== 


Processing user query: search for the current weather in boston

{
│   'query': 'current weather in Boston',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in Boston',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Boston', 'region': 'Massachusetts', 'country': 'United States of America', 'lat': 42.3583, 'lon': -71.0603, 'tz_id': 'America/New_York', 'localtime_epoch': 1746568120, 'localtime': '2025-05-06 17:48'}, 'current': {'last_updated_epoch': 1746567900, 'last_updated': '2025-05-06 17:45', 'temp_c': 12.8, 'temp_f': 55.0, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png', 'code': 1030}, 'wind_mph': 5.1, 'wind_kph': 8.3, 'wind_degree': 132, 'wind_dir': 'SE', 'pressure_mb': 1019.0, 'pressure_in': 30.08, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 100, 'cloud': 100, 'feelslike_c': 12.3, 'feelslike_f': 54.1, 'windchill_c': 12.5, 'windchill_f': 54.5, 'heatindex_c': 13.8, 'heatindex_f': 56.9, 'dewpoint_c': 13.2, 'dewpoint_f': 55.8, 'vis_km': 0.8, 'vis_miles': 0.0, 'uv': 0.6, 'gust_mph': 8.2, 'gust_kph': 13.1}}",
│   │   │   'score': 0.9504822,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Clima de hoy Boston: pronóstico para el 6 de mayo, según National ...',
│   │   │   'url': 'https://libero.pe/estados-unidos/2025/05/06/clima-hoy-boston-pronostico-6-mayo-segun-national-weather-service-pronostico-tiempo-manana-noche-lbeuc-491821',
│   │   │   'content': 'Clima de hoy en Boston: Pronóstico oficial del NWS para este martes, 06 de mayo de 2025 . El martes 06 de mayo, los habitantes de Boston amanecerán con una temperatura de aproximadamente 20',
│   │   │   'score': 0.8684817,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Boston weather in June 2025 | Boston 14 day weather',
│   │   │   'url': 'https://www.weather25.com/north-america/usa/massachusetts/boston?page=month&month=June',
│   │   │   'content': "The temperatures in Boston in June are comfortable with low of 15 ° C and and high up to 24 ° C.. You can expect about 3 to 8 days of rain in Boston during the month of June. It's a good idea to bring along your umbrella so that you don't get caught in poor weather. Our weather forecast can give you a great sense of what weather to expect in Boston in June 2025.",
│   │   │   'score': 0.86303437,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Weather in Boston in June 2025 (Massachusetts) - Detailed Weather ...',
│   │   │   'url': 'https://world-weather.info/forecast/usa/boston/june-2025/',
│   │   │   'content': 'Detailed ⚡ Boston Weather Forecast for June 2025 - day/night 🌡️ temperatures, precipitations - World-Weather.info. Add the current city. Search. Weather; Archive; Widgets °F. World; United States; Massachusetts; Weather in Boston; Weather in Boston in June 2025.',
│   │   │   'score': 0.7724109,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'May 6 Weather in Boston: Detailed Weather Forecast for May 6 in Boston ...',
│   │   │   'url': 'https://world-weather.info/forecast/usa/boston/06-may/',
│   │   │   'content': 'Detailed ⚡ Weather Forecast for May 6 in Boston, Massachusetts, United States - 🌡️ temperature, wind, atmospheric pressure, humidity and precipitations - World-Weather.info ... May 06, 2025 : Atmospheric conditions and temperature °F: RealFeel °F: Atmospheric ... Hourly forecast for 06.05.2024. May 06, 2023 : Atmospheric conditions and',
│   │   │   'score': 0.7159213,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
The current weather in Boston is mostly misty with a temperature of 12.8°C (55°F) and a wind speed of 5.1 mph (8.3 km/h) from the southeast. The humidity is at 100% and the pressure is 1019 mb. There is no precipitation expected today.

========== Query processing completed ========== 



This output shows an AI agent processing a user query about "latest news OpenShift."

* **📍 Step 1: InferenceStep**: The agent *infers* the need to search for information and decides to use the `brave_search` tool with the query "latest news OpenShift."
* **📍 Step 2: ToolExecutionStep**: The agent *executes* the `brave_search` tool. The JSON block is the *output* from this tool, providing search results with titles, URLs, snippets, and relevance scores.
* **📍 Step 3: InferenceStep**: The agent *processes* the search results from the tool and *infers* how to construct a final answer.
* **🤖 Model Response**: This is the AI agent's *final generated response* to the user's query, summarizing the key information extracted from the search results.

Now lets try REact agents

### Creating our first custom tool

This code defines a custom tool called `get_location` for agent to use

* **`@client_tool`**: This decorator from `llama_stack_client.lib.agents.client_tool` marks the `get_location` function as a tool that the agent can discover and use to fulfill user requests. It essentially registers this function within the agent's toolkit.

* **Docstring Notes**: The text within the triple quotes (`"""Docstring goes here"""`) serves as crucial instructions for the AI agent. It explains:
    * **What the tool does**: "Provides the user's location upon request."
    * **How it works**: Mentions using the `geocoder` library and determining location via IP address.
    * **Input parameters**: Describes the `query` parameter.
    * **Output**: Specifies what the tool returns (location information as a string).
    * **Example**: Shows how the tool can be used and its expected output.

The agent reads this docstring to understand the tool's purpose, how to call it (with what arguments), and what kind of result to expect. This allows the agent to strategically use the `get_location` tool when a user asks for their location.

> **Note:**
> This is just one way to create a custom tool, we can also import tools from libraries or use MCP Servers (Again, more on this later...) 
> 

In [6]:
from llama_stack_client.lib.agents.client_tool import client_tool

@client_tool
def get_location(query: str = "location"):
    """
    Provides the user's location upon request.

    This function uses the geocoder library to determine the user's location
    based on their IP address.  It returns the city, state, and country.

    :param query:  The query from the user.  Defaults to "location".
    :type query: str
    :return:  Information about the user's current location.
    :rtype: str

    Example:
        >>> get_location("where am i")
        "Your current location is: Some City, Some State, Some Country"
    """
    import geocoder
    try:
        g = geocoder.ip('me')
        if g.ok:
            return f"Your current location is: {g.city}, {g.state}, {g.country}" # can be modified to return latitude and longitude if needed
        else:
            return "Unable to determine your location"
    except Exception as e:
        return f"Error getting location: {str(e)}"

## Prompt Chaining (Optional)

Prompt chaining allows an AI agent to remember information across multiple turns within the same session. In this example, both user questions share the same `session_id`.

Because they share a session, the agent will remember the answer to the first question ("Where am I?") when interpreting the second question ("search for any weather-related risks in my area..."). This enables the agent to understand "my area" refers to the location identified in the previous turn, leading to a more context-aware and helpful response.

### Defining a Multi-tool Agent

This code initializes an AI agent with specific instructions and tools. 
The `tools` parameter is a list containing elements the agent can use. Here, it includes `get_location` (a custom function defined earlier) and `"builtin::websearch"` (a pre-built web searching capability in Llama Stack). 


The instructions (often refered to as "Prompt" guide the agent on when to use each of these tools based on the user's query.




In [7]:

agent = Agent(
    client, 
    model=SELECTED_MODEL,
    # instructions="""You are a helpful assistant. 
    # When a user asks about their location, you MUST use the get_location tool. When searching for nearby places, you MUST use the websearch tool.
    # """ ,
    instructions="""You are a helpful assistant. 
    When a user asks about their location, you MUST use the get_location tool. When searching for nearby places, you MUST use the brave_search tool.
    """ ,
    tools=[get_location, "builtin::websearch"],
    #sampling_params=sampling_params
)

### inspecting the Agent's response structure with Prompt Chaining

Notice that we are **NOT** generating a new session ID for each question, this way, the convesation memory is shared throughtout the session. 


In [8]:
user_prompts = [
    "Where am I?",
    "search for any weather-related risks in my area that could disrupt network connectivity or system availability?",
]

# Generate a new Unique Identifier for the session, but *NOT* for each question.

new_uuid = uuid.uuid4()
session_id = agent.create_session("prompt-chaining-session-{new_uuid}")

for prompt in user_prompts:
    print("\n"+"="*50)
    print(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )

    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Where am I? blue

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: get_location, Arguments: {'query': 'location'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


'Your current location is: Columbus, Ohio, US'


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
<|python_tag|>{"city": "Columbus", "state": "Ohio", "country": "US"}

========== Query processing completed ========== 


Processing user query: search for any weather-related risks in my area that could disrupt network connectivity or system availability? blue

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'weather risks near Columbus, Ohio, US'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'weather risks near Columbus, Ohio, US',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in Columbus, Ohio, US',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Columbus', 'region': 'Ohio', 'country': 'United States of America', 'lat': 39.9611, 'lon': -82.9989, 'tz_id': 'America/New_York', 'localtime_epoch': 1746566640, 'localtime': '2025-05-06 17:24'}, 'current': {'last_updated_epoch': 1746566100, 'last_updated': '2025-05-06 17:15', 'temp_c': 14.4, 'temp_f': 57.9, 'is_day': 1, 'condition': {'text': 'Light rain', 'icon': '//cdn.weatherapi.com/weather/64x64/day/296.png', 'code': 1183}, 'wind_mph': 6.3, 'wind_kph': 10.1, 'wind_degree': 259, 'wind_dir': 'W', 'pressure_mb': 1015.0, 'pressure_in': 29.96, 'precip_mm': 1.25, 'precip_in': 0.05, 'humidity': 97, 'cloud': 100, 'feelslike_c': 13.9, 'feelslike_f': 57.0, 'windchill_c': 16.8, 'windchill_f': 62.2, 'heatindex_c': 16.9, 'heatindex_f': 62.4, 'dewpoint_c': 11.4, 'dewpoint_f': 52.5, 'vis_km': 4.8, 'vis_miles': 2.0, 'uv': 0.4, 'gust_mph': 11.5, 'gust_kph': 18.6}}",
│   │   │   'score': 0.8034666,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'More rain, thunderstorms possible Tuesday in Columbus, NWS says',
│   │   │   'url': 'https://www.dispatch.com/story/news/local/2025/05/06/columbus-weather-forecast-storms-rain-possible-tuesday-thunderstorm-watch-thunderstorm-warning/83470387007/',
│   │   │   'content': 'Thunderstorms, rain possible Tuesday in Columbus, NWS says Your regularly scheduled rain will proceed as planned Tuesday. After a week of almost non-stop rain and storms, Columbus is in for even more wet weather the evening of Tuesday, May 6. What time will it rain Tuesday in Columbus? Showers and thunderstorms will be possible after 5 p.m. Tuesday. Later, there will be a slight chance of showers between 10 p.m. and midnight, according to an NWS forecast. The forecast does promise some sunny weather as the week turns into the weekend. Here is what to expect from the weather later this week. Columbus weather for the week of May 6, 2025',
│   │   │   'score': 0.79522943,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Columbus, OH Weather Forecast June 2025: Daily Highs/Lows & Rain Trends',
│   │   │   'url': 'https://www.weathertab.com/en/c/o/06/united-states/ohio/columbus/',
│   │   │   'content': 'Explore comprehensive June 2025 weather forecasts for Columbus, including daily high and low temperatures, precipitation risks, and monthly temperature trends. Featuring detailed day-by-day forecasts, dynamic graphs of daily rain probabilities, and temperature trends to help you plan ahead.',
│   │   │   'score': 0.7602419,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Columbus Weather: New week begins with same, wet trend - WSYX ABC 6',
│   │   │   'url': 'https://abc6onyourside.com/news/local/columbus-ohio-weather-forecast-new-week-begins-same-wet-trend-rain-scattered-showers-cloudy-skies-spring-may-4-2025',
│   │   │   'content': 'Columbus Weather: New week begins with same, wet trend Another wet day is on tap as a low-pressure system lingers in Ohio. Scattered rain showers persist today with an isolated thunderstorm possible. While the severe weather threat is low, a stronger storm cannot be ruled out for far East Ohio which is under a Marginal Risk (level 1 out of 5). Temperatures remain cool in the low 60s. Overnight lows will dip into the upper 40s and low 50s under mostly cloudy skies. Keep the umbrellas nearby for the start of the work week as more rain showers slide into the area. The low-pressure system will be kicked out by later Tuesday.',
│   │   │   'score': 0.6884899,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Columbus, Ohio Climate Change Risks and Hazards: Precipitation, Heat',
│   │   │   'url': 'https://climatecheck.com/ohio/columbus',
│   │   │   'content': 'Columbus, OHTop Cli


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Based on the search results, there are several weather-related risks in the Columbus, Ohio area that could disrupt network connectivity or system availability:

1. Light rain and thunderstorms are possible today, with a slight chance of showers between 10 pm and midnight.
2. A low-pressure system is lingering in Ohio, leading to scattered rain showers and isolated thunderstorms.
3. There is a marginal risk (level 1 out of 5) for severe weather in far East Ohio, which could potentially impact network connectivity and system availability.
4. Climate change is increasing the risk of precipitation and heat in Columbus, OH, with projected increases in annual precipitation and an increased risk of extreme precipitation.

It is recommended to take precautions to mitigate these risks, such as keeping umbrellas nearby for rain showers and being aware of potential severe weather threats.

========== Query processing completed =====

## Lab Summary: Interacting with Agents and Tools

In this lab, you took your first steps in understanding and interacting with AI agents using the Llama Stack framework and observe the benefits of Llama Stack in providing a structured and intuitive way to build intelligent agents with access to a variety of tools, both built-in and custom, and the ability to manage conversational memory for more coherent interactions. 

You learned how to:

* **Define a basic AI agent:** You created an agent and provided it with instructions on how to behave.
* **Utilize built-in tools:** You explored how agents can leverage pre-existing tools within Llama Stack, specifically the `websearch` tool, to gather information from the web.
* **Observe agent behavior:** You became familiar with the concepts of "Turns" and "Steps" that an agent goes through when processing a query, including the crucial "Tool Call" step where the agent decides to use a tool.
* **Create custom tools:** You experienced how to extend an agent's capabilities by defining your own specialized tool (`get_location`) using Python and decorators.
* **Implement prompt chaining:** You explored how to maintain context across multiple user interactions within the same session, allowing the agent to remember previous information and use it to answer subsequent questions more effectively.

Through these exercises, you gained a foundational understanding of how agents operate within the Llama Stack ecosystem and how they can be equipped with tools to perform specific tasks. 